# Creating spark context

This script would have been ideal if the results from the approxSimilarityJoin were better. We almost exclusively use pyspark modules, functions and structures to apply the same NLP process as with our python notebook. Unfortunately the only way to record matches and retrieve information from each dataset both sides of the match originated from is to use the approxSimilarityJoin function linked to MinHashLSH and BucketedRandomProjectionLSHModel which yield poor results. Nonetheless we leave the code below for future improvement

In [3]:
import findspark
import os
your_path = os.getcwd()
findspark.init(r'C:\Users\laksa\Desktop\spark-2.4.3-bin-hadoop2.7')

In [4]:
import pyspark 
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

from pyspark.sql import SQLContext
sqlContext = SQLContext(spark)
sc = spark.sparkContext

import pandas as pd
import numpy as np

import pickle
import copy

from pyspark.sql import udf
from pyspark.sql.types import StructType,StringType,StructField,IntegerType
from pyspark.sql.functions import col

from pyspark.ml import Pipeline
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, NGram, Word2Vec, HashingTF, IDF, MinHashLSH, CountVectorizer 

'C:\\Users\\laksa\\Documents\\DEAP\\Pyspark implementation'

# Loading spark dataframes

### Sector schema

In [15]:
sectorSchema = StructType([# StructField('index', IntegerType(), True)\,
                            StructField("Sector", StringType(), True)\
                           ,StructField("Titles", StringType(), True)\
                           ,StructField("text", StringType(), True)])
sectors_pd = pd.read_csv('proto2/articles_full.csv',index_col = 0)
sectors = spark.createDataFrame(sectors_pd,schema=sectorSchema)

#sectors = (spark.read
#    .schema(sectorSchema)
#    .option("header", "true")
#    .csv("proto2/articles_full.csv"))

### Course schema

In [16]:
courseSchema = StructType([# StructField('index', IntegerType(), True)\,
                            StructField("Courses", StringType(), True)\
                           ,StructField("text", StringType(), True)])

courses_pd = pd.read_csv('proto2/courses_full.csv', index_col = 0)
courses = spark.createDataFrame(courses_pd, schema = courseSchema)

#courses = (spark.read
#    .schema(courseSchema)
#    .option("header", "true")
#    .option("mode", "DROPMALFORMED")
#    .csv("proto2/courses_full.csv"))

# Defining pipeline for string similarity

### Defining stopwords for pre-processing (faster than loading from nltk)

In [17]:
stopW = ['i','me','my', 'myself','we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours',
 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its',
 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these',
 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did',
 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about',
 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in',
 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how',
 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same',
 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm',
 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't",
 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan',
 "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't","pdf"]

### Course pipeline 

In [18]:
course = courses.select("text").toDF("text")
sector = sectors.select("text").toDF("text")

In [19]:
model_course = Pipeline(stages=[
    RegexTokenizer(gaps = False, pattern = '\w+', inputCol = 'text', outputCol = 'tokens'),
    StopWordsRemover(stopWords = stopW, inputCol = 'tokens', outputCol = 'tokens_sw'),
    NGram(n=2, inputCol="tokens_sw", outputCol="ngrams"),
    CountVectorizer(inputCol="ngrams", outputCol="rawFeatures"),
    IDF(inputCol="rawFeatures", outputCol="features"),
    MinHashLSH(inputCol="rawFeatures", outputCol="lsh",seed=12)
]).fit(course)

Applying to our data

In [20]:
course_hashed = model_course.transform(course)
sector_hashed = model_course.transform(sector)

In [8]:
course_hashed.show(5)
sector_hashed.show(5)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------+
|                text|              tokens|           tokens_sw|              ngrams|         rawFeatures|            features|             lsh|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------+
|This seminar trie...|[this, seminar, t...|[seminar, tries, ...|[seminar tries, t...|(21196,[820,2567,...|(21196,[820,2567,...| [[1.3137686E7]]|
|This seminar will...|[this, seminar, w...|[seminar, cover, ...|[seminar cover, c...|(21196,[308,499,1...|(21196,[308,499,1...|[[1.36604077E8]]|
|The seminar provi...|[the, seminar, pr...|[seminar, provide...|[seminar provides...|(21196,[25,114,13...|(21196,[25,114,13...| [[6.6699753E7]]|
|The DSS are inter...|[the, dss, are, i...|[dss, interactive...|[dss interactive,...|(21196,[25,31,199...|(21196,[25,31,199...| [[

### Generating dataframe with matches

In [28]:
matches = model_course.stages[-1].approxSimilarityJoin(sector_hashed, course_hashed,threshold = 1000, distCol="EuclideanDistance")
#.select('datasetA','datasetB')

In [29]:
matches.show(100)

Py4JJavaError: An error occurred while calling o1192.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 36.0 failed 1 times, most recent failure: Lost task 0.0 in stage 36.0 (TID 71, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$1: (struct<type:tinyint,size:int,indices:array<int>,values:array<double>>) => array<struct<type:tinyint,size:int,indices:array<int>,values:array<double>>>)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$13$$anon$1.hasNext(WholeStageCodegenExec.scala:636)
	at scala.collection.Iterator$$anon$12.hasNext(Iterator.scala:440)
	at scala.collection.Iterator$JoinIterator.hasNext(Iterator.scala:212)
	at scala.collection.Iterat

In [10]:
matches.count()

Py4JJavaError: An error occurred while calling o488.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 9.0 failed 1 times, most recent failure: Lost task 0.0 in stage 9.0 (TID 23, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$1: (struct<type:tinyint,size:int,indices:array<int>,values:array<double>>) => array<struct<type:tinyint,size:int,indices:array<int>,values:array<double>>>)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$13$$anon$1.hasNext(WholeStageCodegenExec.scala:636)
	at scala.collection.Iterator$$anon$12.hasNext(Iterator.scala:440)
	at scala.collection.Iterator$JoinIterator.hasNext(Iterator.scala:212)
	at scala.collection.Iterator$$anon

## Preparing the data for Graph construction 

### Extracting only text

In [9]:
matches = matches.withColumn("Articles", matches["datasetA"]["text"]).withColumn("AI_topics", matches["datasetB"]["text"]).select("Articles",'AI_topics')

### Matching course labels 

In [10]:
left_join = matches.join(courses, matches.AI_topics == courses.text,how='left') 

# Free space in cache !
matches.unpersist()
courses.unpersist()

left_join = left_join.select("Articles",'Courses')

### Matching sector labels

In [11]:
full_data = left_join.join(sectors, left_join.Articles == sectors.text, how = "left")

# Free space in cache !
left_join.unpersist()
sectors.unpersist()

full_data = full_data.select("Courses","Sector")

### Cleaning the data

In [12]:
full_data = full_data.filter(col('Courses').isNotNull()).filter(col('Sector').isNotNull())

In [13]:
full_data.show(10)

+--------------------+--------------------+
|             Courses|              Sector|
+--------------------+--------------------+
|DT2119 Speech and...|Accomodation and ...|
|DISTRIBUTED ALGOR...|Accomodation and ...|
|       Linear models|Accomodation and ...|
|Optimization for ...|Accomodation and ...|
|Project Managemen...|Accomodation and ...|
|Algebraic Methods...|Accomodation and ...|
|Information Security|Accomodation and ...|
|Computability and...|Accomodation and ...|
|Shape Modeling an...|Accomodation and ...|
|  Probability Theory|Accomodation and ...|
+--------------------+--------------------+
only showing top 10 rows



In [14]:
end = time.time()
print(end-start)

724.6993787288666


### Pandas conversion?

This is probably the most time consuming process of all... If we could somehow avoid having to transfer data as pandas that would be great.

In [15]:
test = full_data.toPandas()
#full_data.unpersist()

In [16]:
test.head()

,Courses,Sector
0,DT2119 Speech and Speaker Recognition,Accomodation and food service activities
1,DISTRIBUTED ALGORITHMS,Accomodation and food service activities
2,Linear models,Accomodation and food service activities
3,Optimization for Data Science,Accomodation and food service activities
4,Project Management and Risk Control,Accomodation and food service activities


In [17]:
test.to_csv('pyspark__results_proto2.csv')

In [ ]:
end = time.time()
print(end-start)